In [ ]:
%pip install pandas
%pip install numpy
%pip install qfinuwa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from qfinuwa import Backtester, Strategy, Indicators, Plotting, API
import pandas as pd
import numpy as np
import os
import math

In [ ]:
# Accesses data files in google drive
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# measures how many standard deviations a value is away from the mean
def zscore(series):
    return (series - series.mean()) / np.std(series)

In [ ]:
stock2 = "VGT"
stock1 = "IYW"

In [ ]:
class MyStrategy(Strategy):

    def __init__(self, quantity=5, period = 15*390, std_open = 1, std_max = 1.5, std_close = 1, std_boost = 0.25):
        self.quantity = quantity
        self.default_quantity = quantity
        self.period = period

        self.std_open = std_open
        self.std_max = std_max
        self.std_close = std_close
        self.std_boost = std_boost

        # Things recorded manually
        self.n_failed_orders = 0  # Number of failed trades
        self.trade_price = 0      # Total price of all trades made
        self.n_trades = 0         # Total trades made
        self.pnl = 0

    def on_data(self, prices, indicators, portfolio):
      if len(prices['close'][stock1]) > self.period:

        ratio = prices['close'][stock1]/prices['close'][stock2]
        ratio_period = ratio[-self.period:]

        if((abs(zscore(ratio_period)[-1]) > self.std_open) and (abs(zscore(ratio_period)[-1]) < self.std_max)):

          if(abs(zscore(ratio_period)[-1]) > (self.std_open + self.std_boost)):
            self.quantity = self.quantity*2

          q = np.sign(zscore(ratio_period)[-1])

          order_success = portfolio.order(stock1, quantity=-q*self.quantity)
          if not order_success: self.n_failed_orders += 1
          else:
            self.trade_price += prices['close'][stock1][-1] * abs(self.quantity)
            self.n_trades += 1
            self.pnl += q * self.quantity * prices['close'][stock1][-1]

          order_success = portfolio.order(stock2, quantity=q*self.quantity)
          if not order_success: self.n_failed_orders += 1
          else:
            self.trade_price += prices['close'][stock2][-1] * abs(self.quantity)
            self.n_trades += 1
            self.pnl += -q * self.quantity * prices['close'][stock2][-1]

        # Otherwise close position
        elif(abs(zscore(ratio_period)[-1]) < self.std_open - self.std_close):     # 0 quantity trades can happen here

          t = portfolio.delta[stock1]
          order_success = portfolio.order(stock1, quantity= -portfolio.delta[stock1])
          if not order_success: self.n_failed_orders += 1
          else:
            self.trade_price += prices['close'][stock1][-1] * abs(t)
            self.n_trades += 1
            self.pnl += t * prices['close'][stock1][-1]

          t = portfolio.delta[stock2]
          order_success = portfolio.order(stock2, quantity= -portfolio.delta[stock2])
          if not order_success: self.n_failed_orders += 1
          else:
            self.trade_price += prices['close'][stock2][-1] * abs(t)
            self.n_trades += 1
            self.pnl += t * prices['close'][stock2][-1]


        self.quantity = self.default_quantity



    def on_finish(self):
          # Added to results object - access using result.on_finish
          return [self.n_failed_orders, self.trade_price, self.n_trades, self.pnl]

In [ ]:
class CustomIndicators(Indicators):
    pass

In [ ]:
# Will need to change the path
backtester = Backtester(MyStrategy, CustomIndicators, [stock1, stock2],
                        data_folder='./drive/MyDrive/Qfin 2023/Data', days='all')

> Precompiling data: 100%|██████████| 169299/169299 [00:01<00:00, 113048.61it/s]


In [ ]:
# Backtester
"""
backtester.fee = 0.002
backtester.starting_balance = 10000
backtester.days = 90
backtester.delta_limit = 1000

rand_seed = 2023

output = backtester.run(cv=50, seed = rand_seed)
output
"""

> Running backtest over 50 samples of 90 days: 100%|██████████| 50/50 [05:40<00:00,  6.81s/it]



{'strategy': {'quantity': 5, 'period': 5850, 'std_open': 1, 'std_max': 1.5, 'std_close': 1, 'std_boost': 0.25}, 'indicator': {}}

Mean ROI:	163514.92814698006
STD ROI:	205554.9452604806

26/01/2022 -> 25/04/2022:	274385.953
14/03/2022 -> 10/06/2022:	357258.125
31/01/2022 -> 29/04/2022:	360032.066
08/12/2021 -> 07/03/2022:	502956.396
14/07/2022 -> 11/10/2022:	344368.891
14/06/2022 -> 09/09/2022:	-78853.088
21/12/2021 -> 18/03/2022:	424332.697
01/08/2022 -> 28/10/2022:	191229.410
30/06/2021 -> 27/09/2021:	32476.122
19/07/2021 -> 15/10/2021:	-25170.779
12/08/2022 -> 09/11/2022:	362074.447
26/11/2021 -> 23/02/2022:	505068.356
16/05/2022 -> 12/08/2022:	-340636.889
12/05/2022 -> 09/08/2022:	-184383.920
24/11/2021 -> 18/02/2022:	388014.699
26/08/2021 -> 23/11/2021:	41001.670
06/10/2021 -> 03/01/2022:	5710.049
25/08/2021 -> 22/11/2021:	57444.225
03/08/2022 -> 31/10/2022:	199657.376
29/04/2021 -> 27/07/2021:	284172.754
20/09/2021 -> 17/12/2021:	-32422.554
10/01/2022 -> 08/04/2022:	170012.369
0